<a href="https://colab.research.google.com/github/tubagokhan/GrinCH/blob/main/RedundancyMeasureRL_ReScore_NGram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import time
import json
import os
import sys
sys.setrecursionlimit(10000000)
print(sys.getrecursionlimit())

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

from google.colab import drive
drive.mount('/content/drive')

!pip install rouge
!pip install sentence_transformers

10000000


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# RL-Redundancy

from rouge import Rouge
from itertools import combinations
from nltk import sent_tokenize

def remove_short_strings(lst, min_length=2):
    return [item for item in lst if len(item) >= min_length]

def rouge_l_f1_score(sentence1, sentence2):
    rouge = Rouge()
    scores = rouge.get_scores(sentence1, sentence2)
    rouge_l = scores[0]['rouge-l']['f']
    return rouge_l


def calculate_redundancy_Rouge_score(sentences):
    if len(sentences) < 2:
        return 0

    summary_sentences = remove_short_strings(sentences)
    total_redundancy_score = 0
    num_pairs = 0

    for sentence1, sentence2 in combinations(summary_sentences, 2):
        rouge_l_score = rouge_l_f1_score(sentence1, sentence2)
        total_redundancy_score += rouge_l_score
        num_pairs += 1

    if num_pairs > 0:
        average_redundancy_score = total_redundancy_score / num_pairs
        return average_redundancy_score
    else:
        return 0


def RLRedundancyScore(corpusList):
    listOfSimilarityDoc=[]
    for doc in corpusList:
        sentences=sent_tokenize(doc)
        docSimilarity=calculate_redundancy_Rouge_score(sentences)
        listOfSimilarityDoc.append(docSimilarity)
        #print(len(listOfSimilarityDoc))
    ReScore=round(sum(listOfSimilarityDoc)/len(listOfSimilarityDoc),2)
    return ReScore

In [9]:
#N-Gram Ratio
from collections import Counter
import re

def calculate_unique_ngram_ratio(text, n):
    # Tokenize the text using NLTK's word_tokenize
    tokens = word_tokenize(text.lower())

    # Generate n-grams using NLTK's ngrams function
    ngrams_list = list(ngrams(tokens, n))

    if not ngrams_list:  # Handle the case where there are no n-grams
        return 0.0

    # Convert n-grams back to strings for counting
    ngrams_strings = [' '.join(gram) for gram in ngrams_list]

    # Calculate the counts of unique and total n-grams
    unique_ngrams_count = len(set(ngrams_strings))
    #print(unique_ngrams_count)
    total_ngrams_count = len(ngrams_strings)
    #print(total_ngrams_count)

    # Calculate the unique n-gram ratio
    unique_ngram_ratio = unique_ngrams_count / total_ngrams_count

    return unique_ngram_ratio

def NGramRedundancyScore(corpusList,n):
    listOfSimilarityDoc=[]
    for doc in corpusList:
        docSimilarity=calculate_unique_ngram_ratio(doc,n)
        listOfSimilarityDoc.append(docSimilarity)
        #print(len(listOfSimilarityDoc))
    ReScore=round(sum(listOfSimilarityDoc)/len(listOfSimilarityDoc),2)
    return ReScore

In [10]:
#ReScore
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from sentence_transformers import SentenceTransformer

import numpy as np

import time

def cosine(u, v):
    return (np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v)))

def redundancyMeasure(corpus):
    sentences=sent_tokenize(corpus)
    length=len(sentences)

    #print("Sentence Count:"+ str(len(sentences)))
    if(len(sentences)<2):
      return 0
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    sentence_embeddings = model.encode(sentences)

    sentenceGraph =np.zeros((length, length))
    for x in range(length):
        for y in range(length):
            similarity= cosine(sentence_embeddings[x],sentence_embeddings[y])
            sentenceGraph[x][y]=abs(similarity)
    sentenceGraph = np.round (sentenceGraph,decimals=2)
    #print(sentenceGraph)

    return mean_exclude_diagonal(sentenceGraph)

def mean_exclude_diagonal(matrix):
    np.fill_diagonal(matrix, np.nan)
    mean_value = np.nanmean(matrix)
    return np.round (mean_value,decimals=2)

def ReScore(corpusList):
    listOfSimilarityDoc=[]
    count=0
    for doc in corpusList:
        docSimilarity=redundancyMeasure(doc)
        listOfSimilarityDoc.append(docSimilarity)
        #print(len(listOfSimilarityDoc))
        count=count+1
        print(count)
    ReScore=round(sum(listOfSimilarityDoc)/len(listOfSimilarityDoc),2)
    return ReScore

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
import json
import os


# Define the file path
file_path = "/content/drive/Othercomputers/My Laptop/DriveAccess/OnlySimpleCTextRank-PubMed-Full.json"  # Replace with the desired file path
#file_path = "/content/drive/Othercomputers/My Laptop/DriveAccess/OnlySimpleClustering-PubMed-Full.json"

with open(file_path, 'r') as file:
    data_list = json.load(file)

print(len(data_list["all_summary"]))

listOfDocuments=data_list["all_summary"]

6658


In [12]:

'''print("###############################################################################")
print("unigram")
print(NGramRedundancyScore(listOfDocuments,1))
print("bigram")
print(NGramRedundancyScore(listOfDocuments,2))
print("trigram")
print(NGramRedundancyScore(listOfDocuments,3))
print("###############################################################################")
print("ReScore")
print(ReScore(listOfDocuments))
print("###############################################################################")'''
print("RL Redundancy SCORE:")
print(RLRedundancyScore(listOfDocuments))
print("###############################################################################")


RL Redundancy SCORE:
0.2
###############################################################################
